# FLUJO DE TRABAJO PARA ESTANDARIZACIÓN DE COLOR Y OBTENCIÓN DE VARIABLES MORFOLÓGICAS Y DE COLOR

## FASE 2

Aplicacion de los parámetros de la carta de color en imágenes nuevas


# DESCRIPTION



In [ ]:
# Import libraries

import cv2
import os
import pandas as pd
import math
#%matplotlib widget
import numpy as np
from plantcv import plantcv as pcv
from collections import Counter
from plantcv.parallel import WorkflowInputs
# Set global debug behavior to None (default), "print" (to file), 
# or "plot" (Jupyter Notebooks or X11)

# HSV and CIE-Lab from RGB values
from ConvRGBtoCIELab import ColorTrans,rgb2lab, rgb_to_hsv 
from utilities import convert_rgb_to_lab_hsv

In [ ]:
# Input/output options IMG_4245
args = WorkflowInputs(
    #images=["img/color_image.jpg"],    
    images=["./img/arandano0001.jpg"],   
    names="image1",
    result="ml_tutorial_results.json",
    outdir="./res",
    writeimg=True,
    debug="plot"
    )

# Set debug to the global parameter 
pcv.params.debug = args.debug

# Change display settings
pcv.params.dpi = 170
pcv.params.text_size = 2
pcv.params.text_thickness = 2



In [ ]:
#Load matrix

loadfname_card = "./params/cardcolor_matrix.npz"

card_matrix = pcv.transform.load_matrix(filename=loadfname_card)

loadfname_std= "./params/std_color_matrix.npz"
std_matrix = pcv.transform.load_matrix(filename=loadfname_std)


In [ ]:
# Read in a color image 

# Inputs:
#   filename - Image file to be read in 
#   mode - Return mode of image; either 'native' (default), 'rgb', 'gray', or 'csv' 
img, path, filename = pcv.readimage(filename=args.image1)



In [ ]:
#Color correct your image to the standard values
#look at the image - does the color look good? If it looks crazy, you probably don't have the card found well and need to go back and define the start and spacing for the card

img_cc = pcv.transform.affine_color_correction(img, card_matrix, std_matrix)
pcv.plot_image(img_cc)


In [ ]:
# Inputs:
#   rbg_img      = original image
#   original_img = whether to include the original RGB images in the display: True (default) or False
colorspace_img = pcv.visualize.colorspaces(rgb_img=img_cc)

In [ ]:
#The good one
channel = pcv.rgb2gray_lab(rgb_img=img_cc, channel='l')


In [ ]:
#th_binary=pcv.threshold.binary(gray_img=k_channel, threshold=115, object_type="light")
th_binary = pcv.threshold.otsu(gray_img=channel, object_type='dark')

In [ ]:
#Eliminación de ruido y agujeros en la máscara de la planta
mask_fill = pcv.fill(bin_img=th_binary, size=200)
mask_fill = pcv.fill_holes(bin_img=mask_fill)


In [ ]:

roi1 = pcv.roi.rectangle(img=img_cc, x=200, y=200, h=1000, w=1000)


In [ ]:

# Make a new filtered mask that only keeps the leaves in your ROI and not objects outside of the ROI
# We have set to partial here so that if a leaf extends outside of your ROI it will still be selected. Switch to "cutto" if you have other plants that are getting selected on accident

# Inputs:
#    mask            = the clean mask you made above
#    roi            = the region of interest you specified above
#    roi_type       = 'partial' (default, for partially inside the ROI), 'cutto', or 
#                     'largest' (keep only largest contour)

kept_mask  = pcv.roi.filter(mask=mask_fill, roi=roi1, roi_type='partial')



In [ ]:
#Se aplica la máscara a la imagen original para obtener solo la superficie de planta
color_planta=pcv.apply_mask(img=img_cc, mask=kept_mask, mask_color='white')

In [ ]:

#Establezca los valores de los parámetros de dimensiónes del chip

chip_length_avg=95.16
ratio_length=0.126097


ratio_area=0.01590057


# Define the column titles
column_titles=['archivo','npenca','altura(mm)','ancho(mm)', 'área(mm2)', 'perímétro(mm)','solidez','redondez',
               'r_plant_prom','g_plant_prom','b_plant_prom',
               'lab-L','lab-a','lab-b','hsv-h','hsv-s','hsv-v']


# Create an empty DataFrame with the specified columns
data = pd.DataFrame(columns=column_titles)


In [ ]:

def mask_and_reshape(img: np.ndarray) -> np.ndarray:
    """
    Applies a filter to keep pixel values strictly between 0 and 255 (excl.) 
    and reshapes the image into a 2D array of shape (num_pixels, 3).
    Returns the filtered (R,G,B) vector.
    """
    # Reshape image to N x 3
    rows, cols, layers = img.shape
    vector = np.reshape(img, (rows * cols, -1))

    # Keep only pixels that are strictly greater than 0 and less than 255
    # in all three channels
    mask = np.all((vector > 0) & (vector < 255), axis=1)
    return vector[mask]


def compute_color_stats(rgb_array: np.ndarray):
    """
    Given a 2D array of shape (N, 3) with (R,G,B) values,
    returns:
      - mean_b, mean_g, mean_r
      - mode_b, mode_g, mode_r
    """
    mean_b = rgb_array[:, 0].mean()
    mean_g = rgb_array[:, 1].mean()
    mean_r = rgb_array[:, 2].mean()


    return (mean_b, mean_g, mean_r)



In [ ]:
# Copy gray image

original = color_planta.copy()
maskedonly = kept_mask.copy()

# Canny edge detection
canny_original = cv2.Canny(original, 120, 255, 1)
kernel = np.ones((5, 5), np.uint8)
dilate_original = cv2.dilate(canny_original, kernel, iterations=1)

# Preprocess masked image
blurred_maskedonly = cv2.GaussianBlur(maskedonly, (3, 3), 0)
canny_maskedonly = cv2.Canny(blurred_maskedonly, 120, 255, 1)
dilate_maskedonly = cv2.dilate(canny_maskedonly, kernel, iterations=1)

# Find contours
cnts = cv2.findContours(dilate_maskedonly, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]


n_plant = 0

for c in cnts:
    list_val = []
    n_plant += 1

    # Basic info
    list_val = [filename, n_plant]

    # Bounding box
    x, y, w, h = cv2.boundingRect(c)
    cv2.rectangle(original, (x, y), (x + w, y + h), (0, 0, 0), 2)

    # Slices of original, masked, etc.
    ROI_orig = original[y:y+h, x:x+w]
    ROI_mask = maskedonly[y:y+h, x:x+w]

    # Visualization (optional)
    pcv.plot_image(ROI_orig)
    pcv.plot_image(ROI_mask)

    # Shape analysis (size, perimeter, solidity, etc.)
    analysis_image = pcv.analyze.size(
        img=ROI_orig,
        labeled_mask=ROI_mask,
        n_labels=n_plant
    )

    height = pcv.outputs.observations['default_1']['height']['value'] * ratio_length
    width = pcv.outputs.observations['default_1']['width']['value'] * ratio_length
    area = pcv.outputs.observations['default_1']['area']['value'] * ratio_area
    perimeter = pcv.outputs.observations['default_1']['perimeter']['value'] * ratio_length
    solidity = pcv.outputs.observations['default_1']['solidity']['value']
    roundness = (4 * math.pi * area) / (perimeter ** 2) if perimeter != 0 else 0

    list_val.extend([height, width, area, perimeter, solidity, roundness])

    # Color analysis on entire ROI
    rgb_plant = mask_and_reshape(ROI_orig)
    b_plant_prom, g_plant_prom, r_plant_prom = compute_color_stats(rgb_plant)

    # Convert to Lab/HSV
    rgb_triplet = (r_plant_prom, g_plant_prom, b_plant_prom)
    lab_color, hsv_color = convert_rgb_to_lab_hsv(rgb_triplet)
    lab = lab_color.tolist()
    hsv = hsv_color.tolist()

    list_val.extend([
        r_plant_prom, g_plant_prom, b_plant_prom,
        lab[0], lab[1], lab[2],
        hsv[0], hsv[1], hsv[2]
    ])

    # Finally, add row to DataFrame
    data.loc[len(data)] = list_val




In [ ]:
#Opcional para ver los objetos y resultados del análisis con la función pcv.analyze.size 
# Save out data to file
pcv.outputs.save_results(filename="./res/results_analyze_size.txt", outformat="json")

In [ ]:
#Excel file
archivo='./res/res_escanner.xlsx'

# Create the directory if it doesn't exist
directory = os.path.dirname(archivo)
if directory and not os.path.exists(directory):
    os.makedirs(directory, exist_ok=True)

# Convert some columns to integers
#data[['R','G','B','moda_r','moda_g','moda_b']] = data[['R','G','B','moda_r','moda_g','moda_b']].astype(int)


# Check if the file exists
if os.path.exists(archivo):
    # If the file exists, load the workbook and append to the 'Results' sheet if it exists
    with pd.ExcelWriter(archivo, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        # Check if 'Results' sheet already exists
        if 'Results' in writer.book.sheetnames:
            # Get the maximum row in the existing sheet to append the new data below it
            startrow = writer.sheets['Results'].max_row
        else:
            # If the 'Results' sheet does not exist, start from the first row
            startrow = 0
        # Write the DataFrame to the existing file, appending data if the sheet exists
        data.to_excel(writer, sheet_name='Results', header=startrow == 0, startrow=startrow, index=True)
else:
    # If the file does not exist, create it and write the data
    with pd.ExcelWriter(archivo, engine='openpyxl') as writer:
        data.to_excel(writer, sheet_name='Results', index=True)
